## End-to-End Byte Pair Encoding (BPE) Implementation

### Introduction: The Need for Subword Tokenization and BPE

**What is Tokenization?**
In Natural Language Processing (NLP), tokenization is the fundamental first step of breaking down raw text into smaller units called tokens. These tokens are the basic building blocks that machine learning models process (e.g., words, punctuation marks).

**Limitations of Simple Word Tokenization:**
Simply splitting text by spaces or punctuation seems intuitive but faces major challenges:
1.  **Vocabulary Explosion:** If every unique word is a token, the vocabulary size can become enormous, especially with large corpora containing many word variations (e.g., 'run', 'runs', 'running', 'ran', 'runner'). This requires significant memory and computational resources.
2.  **Out-of-Vocabulary (OOV) Problem:** Models trained on a fixed vocabulary cannot handle words they haven't seen during training (e.g., new proper nouns, technical terms, misspellings, newly coined words). These OOV words are typically mapped to a generic `<UNK>` (unknown) token, losing valuable information.
3.  **Morphological Relationships:** Word-level tokenization ignores the relationships between words sharing common roots or affixes (e.g., 'running', 'runner').

**Subword Tokenization as a Solution:**
Subword tokenization algorithms aim to find a middle ground between character-level (tiny vocabulary, very long sequences) and word-level tokenization. They break words into smaller, meaningful units (subwords or morphemes). 
*   Common words might remain single tokens (e.g., 'the').
*   Less common words are broken down (e.g., 'tokenization' -> 'token', 'ization').
*   Rare or unknown words are broken down further, potentially to individual characters (e.g., 'BPEology' -> 'B', 'P', 'E', 'ology' or even smaller pieces depending on training).
This approach keeps the vocabulary size manageable while drastically reducing the OOV problem and preserving some morphological information.

**Byte Pair Encoding (BPE): The Algorithm**
BPE, originally a data compression technique, was adapted for NLP tokenization. Its core idea is simple and elegant: **iteratively merge the most frequent pair of adjacent symbols (characters or previously merged subwords) in the training corpus.**

**High-Level BPE Steps:**
1.  **Initialization:** Start with a vocabulary containing only individual characters found in the training text. Prepare the text by splitting it into words and representing each word as a sequence of its characters plus a special end-of-word marker (essential for learning word boundaries).
2.  **Training (Iterative Merging):** Repeat the following for a predefined number of steps (merges):
    a.  Count the frequency of all adjacent pairs of symbols across the entire corpus (considering word frequencies).
    b.  Find the symbol pair that occurs most frequently.
    c.  Merge this pair into a single new symbol (subword).
    d.  Add this new symbol to the vocabulary.
    e.  Replace all occurrences of the original pair in the corpus representation with the new symbol.
3.  **Tokenization (Applying Learned Rules):** To tokenize new text:
    a.  Preprocess it in the same way as the training data (split words, add end-of-word markers).
    b.  Apply the learned merge operations *in the exact order they were learned* (highest priority first). Greedily merge pairs within each word until no more learned merges can be applied.

**This Notebook's Approach:**
We will implement this entire process step-by-step, inline, without using Python functions or classes. Each conceptual step will be broken down into the smallest possible code block, accompanied by detailed theory, as requested.

### Step 0: Setup - Libraries and Corpus Definition

**Goal:** Prepare the environment by importing necessary tools and defining the text data we will work with.

#### Step 0.1: Import Libraries

**Theory:** We need basic Python libraries:
*   `re`: The regular expression library, primarily used here for splitting the raw text into initial word/token units.
*   `collections`: Specifically, we use `collections.Counter` for efficiently counting the frequencies of words and symbol pairs, which is central to BPE.

In [18]:
# Import necessary standard libraries
import re 
import collections

print("Libraries 're' and 'collections' imported.")

Libraries 're' and 'collections' imported.


#### Step 0.2: Define the Training Corpus

**Theory:** BPE is a data-driven algorithm. It learns merge rules based on the patterns present in a training corpus. We will use a small excerpt from Lewis Carroll's "Alice's Adventures in Wonderland" as our training data. A larger, more diverse corpus would result in a more general-purpose tokenizer, but this smaller example allows us to trace the process more easily.

In [ ]:
# Define the raw text corpus for training the BPE tokenizer
corpus_raw = """
Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversation?'
So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.
"""

print(f"Training corpus defined (length: {len(corpus_raw)} characters).")

Training corpus defined (length: 593 characters).


### Step 1: Preprocessing and Initialization

**Goal:** Transform the raw text corpus into the initial state required for the BPE training algorithm. This involves standardizing the text, splitting it into manageable units, counting frequencies, representing these units as symbol sequences, and defining the starting vocabulary.

#### Step 1.1: Lowercase the Corpus

**Theory:** Converting the entire corpus to lowercase ensures that the same word, regardless of its capitalization (e.g., 'Alice' vs. 'alice'), is treated as the same unit during frequency counting and merging. This simplifies the process and prevents learning separate tokens for capitalized variations unless specifically desired (which would require a different preprocessing strategy).

In [20]:
# Convert the raw corpus to lowercase
corpus_lower = corpus_raw.lower()

# Optional: Display a snippet of the lowercased corpus
# print("Lowercased corpus snippet:")
# print(corpus_lower[:100]) 
print("Corpus converted to lowercase.")

Corpus converted to lowercase.


#### Step 1.2: Initial Word/Token Splitting

**Theory:** We need to split the text into its basic constituents. While BPE operates on subwords, it typically starts by considering word-level units (including punctuation). We use a regular expression `r'\w+|[^\s\w]+'` via `re.findall`:
*   `\w+`: Matches one or more alphanumeric characters (letters, numbers, and underscore). This captures standard words.
*   `|`: Acts as an OR operator.
*   `[^\s\w]+`: Matches one or more characters that are *not* whitespace (`\s`) and *not* word characters (`\w`). This captures punctuation marks like commas, periods, colons, quotes, parentheses, etc., as separate tokens.
The result is a list of strings, where each string is either a word or a punctuation mark.

In [37]:
# Define the regular expression for splitting words and punctuation
split_pattern = r'\w+|[^\s\w]+'

# Apply the regex to the lowercased corpus to get a list of initial tokens
initial_word_list = re.findall(split_pattern, corpus_lower)

print(f"Corpus split into {len(initial_word_list)} initial words/tokens.")
# Optional: Display the first few tokens
print(f"First 3 initial tokens: {initial_word_list[:3]}")

Corpus split into 127 initial words/tokens.
First 3 initial tokens: ['alice', 'was', 'beginning']


#### Step 1.3: Calculate Word/Token Frequencies

**Theory:** The core principle of BPE is merging the *most frequent* pairs. Therefore, we must know how often each unique initial word/token appears in the corpus. Pairs found within more frequent words will have a higher impact on the merge decisions. `collections.Counter` efficiently creates a dictionary-like object mapping each unique item (word/token) to its count.

In [39]:
# Use collections.Counter to count frequencies of items in initial_word_list
word_frequencies = collections.Counter(initial_word_list)

print(f"Calculated frequencies for {len(word_frequencies)} unique words/tokens.")
# Display the 3 most frequent tokens and their counts
print("3 Most frequent tokens:")
for token, count in word_frequencies.most_common(3):
    print(f"  '{token}': {count}")

Calculated frequencies for 86 unique words/tokens.
3 Most frequent tokens:
  'the': 7
  'of': 5
  'her': 5


#### Step 1.4: Initial Corpus Representation (Character Split with `</w>`)

**Theory:** BPE training operates on sequences of symbols. We need to transform our list of unique words/tokens into this format. For each unique word/token:
1.  Split it into a list of individual characters.
2.  Append a special end-of-word symbol (we'll use `</w>`) to this list.
This `</w>` marker is critically important:
*   **Boundary Detection:** It prevents BPE from merging characters across different words. For example, the 's' at the end of 'apples' should not merge with the 'a' at the beginning of 'and'.
*   **Learning Word Endings:** It allows the algorithm to learn common word endings as distinct subword units (e.g., 'ing</w>', 'ed</w>', 's</w>').
We store this mapping (original word -> list of symbols) in a dictionary.

In [40]:
# Define the special end-of-word symbol
end_of_word_symbol = '</w>'

# Create a dictionary to hold the initial representation of the corpus
# Key: original unique word/token, Value: list of characters + end_of_word_symbol
initial_corpus_representation = {}

# Iterate through the unique words/tokens identified by the frequency counter
for word in word_frequencies:
    # Convert the word string into a list of its characters
    char_list = list(word)
    # Append the end-of-word symbol to the list
    char_list.append(end_of_word_symbol)
    # Store this list in the dictionary with the original word as the key
    initial_corpus_representation[word] = char_list

print(f"Created initial corpus representation for {len(initial_corpus_representation)} unique words/tokens.")
# Optional: Display the representation for a sample word
example_word = 'beginning'
if example_word in initial_corpus_representation:
    print(f"Representation for '{example_word}': {initial_corpus_representation[example_word]}")
example_punct = '.'
if example_punct in initial_corpus_representation:
    print(f"Representation for '{example_punct}': {initial_corpus_representation[example_punct]}")

Created initial corpus representation for 86 unique words/tokens.
Representation for 'beginning': ['b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g', '</w>']
Representation for '.': ['.', '</w>']


#### Step 1.5: Build Initial Character Vocabulary

**Theory:** The BPE algorithm starts its vocabulary with the set of all individual symbols present in the initial corpus representation. This includes all unique characters from the original text *plus* the special `</w>` symbol we added. Using a Python `set` automatically handles uniqueness – adding an existing character multiple times has no effect.

In [41]:
# Initialize an empty set to store the unique initial symbols (vocabulary)
initial_vocabulary = set()

# Iterate through the character lists stored in the initial corpus representation
for word in initial_corpus_representation:
    # Get the list of symbols for the current word
    symbols_list = initial_corpus_representation[word]
    # Update the vocabulary set with the symbols from this list
    # The `update` method adds all elements from an iterable (like a list) to the set
    initial_vocabulary.update(symbols_list)

# Although update should have added '</w>', we can explicitly add it for certainty
# initial_vocabulary.add(end_of_word_symbol)

print(f"Initial vocabulary created with {len(initial_vocabulary)} unique symbols.")
# Optional: Display the sorted list of initial vocabulary symbols
print(f"Initial vocabulary symbols: {sorted(list(initial_vocabulary))}")

Initial vocabulary created with 31 unique symbols.
Initial vocabulary symbols: ["'", '(', ')', ',', '-', '.', ':', '</w>', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


### Step 2: BPE Training - Iterative Merging

**Goal:** This is the core BPE learning phase. We will iteratively find the most frequent adjacent pair of symbols in our current corpus representation and merge them into a new, single symbol (subword). This process builds the subword vocabulary and the ordered list of merge rules.

#### Step 2.1: Initialize Training State Variables

**Theory:** Before starting the loop, we need:
1.  `num_merges`: To define how many merge operations we want to perform. This directly controls the final vocabulary size (initial size + `num_merges`). Choosing this value is a hyperparameter; larger values capture more complex subwords but increase vocabulary size.
2.  `learned_merges`: An empty dictionary to store the merge rules as we learn them. The key will be the pair tuple (e.g., `('t', 'h')`), and the value will be the merge priority (an integer, starting from 0, indicating *when* the merge was learned). Lower numbers mean higher priority.
3.  `current_corpus_split`: A variable to hold the state of the corpus representation as it gets modified by merges in each iteration. We initialize it as a *copy* of the `initial_corpus_representation` to avoid modifying the original starting point.
4.  `current_vocab`: A variable to hold the vocabulary as it grows with new merged symbols. We initialize it as a *copy* of the `initial_vocabulary`.

In [25]:
# Define the desired number of merge operations
# This determines how many new subword tokens will be added to the initial character vocab
num_merges = 75 # Let's use 75 merges for this example

# Initialize an empty dictionary to store the learned merge rules
# Format: { (symbol1, symbol2): merge_priority_index }
learned_merges = {}

# Create a working copy of the corpus representation to modify during training
# Using .copy() ensures we don't alter the original initial_corpus_representation
current_corpus_split = initial_corpus_representation.copy()

# Create a working copy of the vocabulary to modify during training
current_vocab = initial_vocabulary.copy()

print(f"Training state initialized. Target number of merges: {num_merges}")
print(f"Initial vocabulary size: {len(current_vocab)}")

Training state initialized. Target number of merges: 75
Initial vocabulary size: 31


#### Step 2.2: The Main Training Loop Structure

**Theory:** We will now iterate `num_merges` times. Inside this loop, we perform the core BPE steps: count pairs, find the best pair, store the rule, create the new symbol, update the corpus representation, and update the vocabulary.

In [26]:
# Start the main loop that iterates for the specified number of merges
print(f"\n--- Starting BPE Training Loop ({num_merges} iterations) ---")
for i in range(num_merges):
    # --- Code for steps 2.3 to 2.9 will go inside this loop --- 
    # Print the current iteration number (starting from 1)
    print(f"\nIteration {i + 1}/{num_merges}")

    # --- Step 2.3 (Inside Loop): Calculate Pair Statistics --- 
    # Theory: We must recalculate pair frequencies in *every* iteration because the 
    # corpus representation changes after each merge. A pair that was frequent might 
    # become less frequent after its components are merged into a new symbol.
    # We iterate through each unique *original* word and its frequency. For each word,
    # we get its *current* symbol representation from `current_corpus_split`. We then
    # iterate through adjacent symbols in that representation and increment the count
    # for that pair in `pair_counts`, weighted by the original word's frequency.
    print("  Step 2.3: Calculating pair statistics...")
    pair_counts = collections.Counter()
    # Iterate through the original words and their frequencies
    for word, freq in word_frequencies.items():
        # Get the *current* list of symbols for this word from the evolving representation
        symbols = current_corpus_split[word]
        # Iterate through the adjacent pairs in the current symbol list
        # Loop from the first symbol up to the second-to-last symbol
        for j in range(len(symbols) - 1):
            # Form the pair (tuple of two adjacent symbols)
            pair = (symbols[j], symbols[j+1])
            # Increment the count for this pair by the frequency of the original word
            pair_counts[pair] += freq 
    print(f"  Calculated frequencies for {len(pair_counts)} unique pairs.")
    # Optional: print top 5 pairs found in this iteration
    # if pair_counts:
    #     print(f"    Top 5 pairs this iteration: {pair_counts.most_common(5)}")

    # --- Step 2.4 (Inside Loop): Check for Termination Condition ---
    # Theory: If, after counting, `pair_counts` is empty, it means there are no 
    # adjacent pairs left in any word representation in the corpus. This can happen 
    # if the corpus is very small or if all possible merges have effectively been done.
    # In this case, we can't proceed, so we break out of the training loop early.
    print("  Step 2.4: Checking if pairs exist...")
    if not pair_counts:
        print("  No more pairs found to merge. Stopping training loop early.")
        break # Exit the 'for i in range(num_merges)' loop
    print("  Pairs found, continuing training.")

    # --- Step 2.5 (Inside Loop): Find the Best Pair --- 
    # Theory: We need to find the key (the pair tuple) in the `pair_counts` counter 
    # that has the maximum associated value (the frequency count). The `max()` function 
    # can take a `key` argument, which specifies a function to determine the sorting/comparison value.
    # `pair_counts.get` is a function that returns the value for a given key in the counter.
    # So, `max(pair_counts, key=pair_counts.get)` finds the pair with the highest count.
    print("  Step 2.5: Finding the most frequent pair...")
    try:
        best_pair = max(pair_counts, key=pair_counts.get)
        best_pair_frequency = pair_counts[best_pair]
        print(f"  Found best pair: {best_pair} with frequency {best_pair_frequency}")
    except ValueError:
        # This should theoretically be caught by the 'if not pair_counts' check above,
        # but adding robust error handling is good practice.
        print("  Error: Could not find maximum in empty pair_counts. Stopping.")
        break

    # --- Step 2.6 (Inside Loop): Store Merge Rule ---
    # Theory: We record which pair was chosen to be merged in this iteration. The priority
    # is simply the iteration index `i`. This ordered list of merges is crucial for 
    # correctly tokenizing new text later.
    print(f"  Step 2.6: Storing merge rule (Priority: {i})...")
    learned_merges[best_pair] = i
    print(f"  Stored: {best_pair} -> Priority {i}")

    # --- Step 2.7 (Inside Loop): Create New Symbol ---
    # Theory: The new symbol representing the merged pair is created by simply 
    # concatenating the string representations of the two symbols in the pair.
    print("  Step 2.7: Creating new symbol from best pair...")
    new_symbol = "".join(best_pair)
    print(f"  New symbol created: '{new_symbol}'")

    # --- Step 2.8 (Inside Loop): Update Corpus Representation ---
    # Theory: This is the most complex part of the loop. We must go through every
    # word representation in `current_corpus_split` and replace *all* occurrences
    # of the `best_pair` sequence with the `new_symbol`. It's essential to create
    # a *new* dictionary (`next_corpus_split`) to store these results, rather than
    # modifying `current_corpus_split` in place while iterating over it. 
    # For each word, we scan its current symbol list (`old_symbols`). We build a 
    # `new_symbols` list. If we encounter the `best_pair` starting at index `k`, 
    # we append `new_symbol` to `new_symbols` and advance our scan position `k` by 2.
    # Otherwise, we just append the symbol `old_symbols[k]` and advance `k` by 1.
    # After processing all words, `current_corpus_split` is updated to become 
    # `next_corpus_split` for the *next* training iteration.
    print("  Step 2.8: Updating corpus representation...")
    next_corpus_split = {}
    # Iterate through all original words (keys in the current split dictionary)
    for word in current_corpus_split:
        # Get the list of symbols for this word *before* applying the current merge
        old_symbols = current_corpus_split[word]
        # Initialize an empty list to build the new sequence of symbols for this word
        new_symbols = []
        # Initialize scan index for the old_symbols list
        k = 0
        # Scan through the old symbols list
        while k < len(old_symbols):
            # Check if we are not at the very last symbol (to allow pair formation)
            # and if the pair starting at index k matches the best_pair to be merged
            if k < len(old_symbols) - 1 and (old_symbols[k], old_symbols[k+1]) == best_pair:
                # If match found, append the new merged symbol to our new list
                new_symbols.append(new_symbol)
                # Advance the scan index by 2 (skipping both parts of the merged pair)
                k += 2
            else:
                # If no match, just append the current symbol from the old list
                new_symbols.append(old_symbols[k])
                # Advance the scan index by 1
                k += 1
        # Store the newly constructed symbol list for this word in the temporary dictionary
        next_corpus_split[word] = new_symbols
        
    # After processing all words, update the main corpus split to reflect the merge
    current_corpus_split = next_corpus_split
    print("  Corpus representation updated for all words.")
    # Optional: Show how a specific word changed after this merge
    # if 'beginning' in current_corpus_split:
    #     print(f"    Example 'beginning' now: {current_corpus_split['beginning']}")

    # --- Step 2.9 (Inside Loop): Update Vocabulary ---
    # Theory: Add the newly created `new_symbol` (the result of the merge)
    # to the `current_vocab` set. Sets automatically handle duplicates.
    print("  Step 2.9: Updating vocabulary...")
    current_vocab.add(new_symbol)
    print(f"  Added '{new_symbol}' to vocabulary. Current size: {len(current_vocab)}")

# --- Step 2.10: Training Loop Finished ---
# Theory: Once the loop completes (either reaches `num_merges` or breaks early),
# the training process is finished. The final learned state is contained in 
# `learned_merges`, `current_vocab`, and `current_corpus_split`.
print(f"\n--- BPE Training Loop Finished after {i + 1} iterations (or target reached) ---")

# Assign final state variables for clarity (optional, could just use current_*)
final_vocabulary = current_vocab
final_learned_merges = learned_merges
final_corpus_representation = current_corpus_split

print("Final vocabulary, merge rules, and corpus representation are ready.")


--- Starting BPE Training Loop (75 iterations) ---

Iteration 1/75
  Step 2.3: Calculating pair statistics...
  Calculated frequencies for 156 unique pairs.
  Step 2.4: Checking if pairs exist...
  Pairs found, continuing training.
  Step 2.5: Finding the most frequent pair...
  Found best pair: ('e', '</w>') with frequency 21
  Step 2.6: Storing merge rule (Priority: 0)...
  Stored: ('e', '</w>') -> Priority 0
  Step 2.7: Creating new symbol from best pair...
  New symbol created: 'e</w>'
  Step 2.8: Updating corpus representation...
  Corpus representation updated for all words.
  Step 2.9: Updating vocabulary...
  Added 'e</w>' to vocabulary. Current size: 32

Iteration 2/75
  Step 2.3: Calculating pair statistics...
  Calculated frequencies for 161 unique pairs.
  Step 2.4: Checking if pairs exist...
  Pairs found, continuing training.
  Step 2.5: Finding the most frequent pair...
  Found best pair: ('i', 'n') with frequency 16
  Step 2.6: Storing merge rule (Priority: 1)...
  Sto

### Step 3: Inspect Training Results

**Goal:** Examine the artifacts produced by the BPE training process to understand what was learned.

#### Step 3.1: Final Vocabulary Size

**Theory:** The final vocabulary contains all initial characters plus all the new subword symbols created during the merge process. Its size is a key outcome of the training.

In [27]:
print("--- Inspecting Training Results ---")
print(f"Final Vocabulary Size: {len(final_vocabulary)} tokens")

--- Inspecting Training Results ---
Final Vocabulary Size: 106 tokens


#### Step 3.2: Learned Merge Rules (Sorted by Priority)

**Theory:** The `final_learned_merges` dictionary contains the crucial information about *which* pairs were merged and *in what order* (priority). To understand the process and apply it correctly during tokenization, it's helpful to view these merges sorted by their priority value (the iteration index `i`).

In [28]:
print("\nLearned Merge Rules (Sorted by Priority):")

# Convert the dictionary items to a list of (pair, priority) tuples
merges_list = list(final_learned_merges.items())

# Sort the list based on the priority (the second element of the tuple, index 1)
# `lambda item: item[1]` tells sort to use the priority value for sorting
sorted_merges_list = sorted(merges_list, key=lambda item: item[1])

# Display the sorted merges
print(f"Total merges learned: {len(sorted_merges_list)}")
# Print a sample if the list is long, otherwise print all
display_limit = 20 
if len(sorted_merges_list) <= display_limit * 2:
    for pair, priority in sorted_merges_list:
        print(f"  Priority {priority}: {pair} -> '{''.join(pair)}'")
else:
    print("  (Showing first 10 and last 10 merges)")
    # Print first N
    for pair, priority in sorted_merges_list[:display_limit // 2]:
        print(f"  Priority {priority}: {pair} -> '{''.join(pair)}'")
    print("  ...")
    # Print last N
    for pair, priority in sorted_merges_list[-display_limit // 2:]:
        print(f"  Priority {priority}: {pair} -> '{''.join(pair)}'")


Learned Merge Rules (Sorted by Priority):
Total merges learned: 75
  (Showing first 10 and last 10 merges)
  Priority 0: ('e', '</w>') -> 'e</w>'
  Priority 1: ('i', 'n') -> 'in'
  Priority 2: ('e', 'r') -> 'er'
  Priority 3: ('t', 'h') -> 'th'
  Priority 4: ('d', '</w>') -> 'd</w>'
  Priority 5: ('s', '</w>') -> 's</w>'
  Priority 6: ('in', 'g') -> 'ing'
  Priority 7: ('ing', '</w>') -> 'ing</w>'
  Priority 8: ('t', '</w>') -> 't</w>'
  Priority 9: ('y', '</w>') -> 'y</w>'
  ...
  Priority 65: ('picture', 's</w>') -> 'pictures</w>'
  Priority 66: ('con', 'ver') -> 'conver'
  Priority 67: ('conver', 's') -> 'convers'
  Priority 68: ('convers', 'a') -> 'conversa'
  Priority 69: ('conversa', 'ti') -> 'conversati'
  Priority 70: ('s', 'e</w>') -> 'se</w>'
  Priority 71: ('th', 'ou') -> 'thou'
  Priority 72: ('w', 'i') -> 'wi'
  Priority 73: ('n', '</w>') -> 'n</w>'
  Priority 74: ('l', '</w>') -> 'l</w>'


#### Step 3.3: Final Representation of Example Words

**Theory:** It's useful to see how specific words from the training corpus are represented *after* all the merge operations have been applied. This shows the final token sequence for known words according to the learned BPE rules.

In [29]:
print("\nFinal Representation of Example Words from Training Corpus:")

# List some words we expect to see interesting tokenization for
example_words_to_inspect = ['beginning', 'conversations', 'sister', 'pictures', 'reading', 'alice']

for word in example_words_to_inspect:
    if word in final_corpus_representation:
        print(f"  '{word}': {final_corpus_representation[word]}")
    else:
        print(f"  '{word}': Not found in original corpus (should not happen if chosen from corpus).")


Final Representation of Example Words from Training Corpus:
  'beginning': ['b', 'e', 'g', 'in', 'n', 'ing</w>']
  'conversations': ['conversati', 'on', 's</w>']
  'sister': ['sister</w>']
  'pictures': ['pictures</w>']
  'reading': ['re', 'ad', 'ing</w>']
  'alice': ['alice</w>']


### Step 4: Tokenization of New Text using Learned Rules

**Goal:** Apply the BPE rules learned during training (`final_learned_merges`) to segment a new, unseen piece of text into a sequence of tokens (characters and subwords from `final_vocabulary`).

#### Step 4.1: Define New Text Input

**Theory:** We need a sample sentence or text that the BPE model hasn't necessarily seen during training to demonstrate its ability to tokenize new input, potentially including words not in the original corpus.

In [30]:
# Define a new text string to tokenize using the learned BPE rules
# This text contains words seen in training ('alice', 'pictures') 
# and potentially unseen words or variations ('tiresome', 'thought')
new_text_to_tokenize = "Alice thought reading was tiresome without pictures."

print(f"--- Tokenizing New Text ---")
print(f"Input Text: '{new_text_to_tokenize}'")

--- Tokenizing New Text ---
Input Text: 'Alice thought reading was tiresome without pictures.'


#### Step 4.2: Preprocess New Text

**Theory:** It is absolutely critical that the new text undergoes the *exact same* initial preprocessing steps as the training data. This includes lowercasing and splitting using the same method (here, the same regular expression).

In [31]:
print("Step 4.2: Preprocessing the new text...")
# 1. Lowercase the new text
new_text_lower = new_text_to_tokenize.lower()
print(f"  Lowercased: '{new_text_lower}'")

# 2. Split into words/tokens using the same regex as in training
# Recall: split_pattern = r'\w+|[^\s\w]+'
new_words_list = re.findall(split_pattern, new_text_lower)
print(f"  Split into words/tokens: {new_words_list}")

Step 4.2: Preprocessing the new text...
  Lowercased: 'alice thought reading was tiresome without pictures.'
  Split into words/tokens: ['alice', 'thought', 'reading', 'was', 'tiresome', 'without', 'pictures', '.']


#### Step 4.3: Prepare for Tokenization Output

**Theory:** We initialize an empty list that will accumulate the final sequence of tokens for the entire input text as we process each word individually.

In [32]:
# Initialize an empty list to store the final sequence of tokens for the whole text
tokenized_output = []
print("Step 4.3: Initialized empty list for tokenized output.")

Step 4.3: Initialized empty list for tokenized output.


#### Step 4.4: Iterate Through Words in New Text

**Theory:** We now process each word/token obtained from the preprocessing step (`new_words_list`) one by one. For each word, we will apply the learned BPE merge rules.

In [33]:
print("Step 4.4: Starting iteration through words of the new text...")
# Loop through each preprocessed word/token from the new text
for word in new_words_list:
    print(f"\n  Processing Word: '{word}'")
    
    # --- Steps 4.5 to 4.7 will go inside this loop --- 
    
    # --- Step 4.5 (Inside Word Loop): Initialize Word Symbols ---
    # Theory: Just like in training initialization, represent the current word as 
    # a list of its characters plus the end-of-word symbol. This is the starting 
    # point for applying merges to this specific word.
    print("    Step 4.5: Initializing symbols for this word...")
    word_symbols = list(word) + [end_of_word_symbol] # Recall: end_of_word_symbol = '</w>'
    print(f"      Initial symbols: {word_symbols}")
    
    # --- Step 4.6 (Inside Word Loop): Inner Loop for Applying Merges ---
    # Theory: This is the core tokenization logic for a *single* word. We need to 
    # repeatedly apply the learned merge rules from `final_learned_merges` to the 
    # `word_symbols` list. The crucial rule is: in each pass, find *all* possible 
    # merges that apply to the current `word_symbols`, but only execute the one 
    # that has the *highest priority* (i.e., the lowest merge index stored in 
    # `final_learned_merges`). After applying that single best merge, the `word_symbols` 
    # list is updated, and we *repeat* the process: scan the *new* list again for 
    # the highest-priority applicable merge. This continues until a pass completes 
    # where no learned merges can be applied to the current `word_symbols`.
    print("    Step 4.6: Applying learned merges iteratively...")
    while True: # Loop until no more merges can be applied to this word
        # --- Find the best merge for the CURRENT pass --- 
        # Initialize variables to track the best merge found *in this pass*
        best_priority_found_this_pass = float('inf') # Use infinity to ensure any valid priority is lower
        pair_to_merge_this_pass = None
        merge_location_this_pass = -1
        
        # Scan through the current word_symbols list to find applicable merges
        # Iterate up to the second-to-last symbol to form pairs
        scan_index = 0
        while scan_index < len(word_symbols) - 1:
            # Form the adjacent pair at the current scan index
            current_pair = (word_symbols[scan_index], word_symbols[scan_index + 1])
            
            # Check if this pair exists in our learned merge rules
            if current_pair in final_learned_merges:
                # If it exists, get its priority (when it was learned)
                current_pair_priority = final_learned_merges[current_pair]
                
                # Check if this pair's priority is better (lower number) than the best found so far *in this pass*
                if current_pair_priority < best_priority_found_this_pass:
                    # If yes, update the best merge found for this pass
                    best_priority_found_this_pass = current_pair_priority
                    pair_to_merge_this_pass = current_pair
                    merge_location_this_pass = scan_index # Record where the best merge starts
                    
            # Move to the next position to check the next pair
            scan_index += 1
            
        # --- Apply the best merge found (if any) or break --- 
        # After scanning the entire current word_symbols list:
        if pair_to_merge_this_pass is not None:
            # An applicable merge was found. Apply the one with the highest priority.
            merged_symbol = "".join(pair_to_merge_this_pass)
            print(f"      Applying highest priority merge: {pair_to_merge_this_pass} (Priority {best_priority_found_this_pass}) at index {merge_location_this_pass} -> '{merged_symbol}'")
            
            # Reconstruct the word_symbols list with the merge applied
            # Slice before the merge + new symbol + slice after the merge
            word_symbols = word_symbols[:merge_location_this_pass] + [merged_symbol] + word_symbols[merge_location_this_pass + 2:]
            print(f"      Updated symbols: {word_symbols}")
            # Continue to the next iteration of the 'while True' loop to scan the *updated* symbols list
        else:
            # No applicable merges were found in the entire scan of the current symbols list
            print("      No more applicable learned merges found for this word.")
            break # Exit the 'while True' loop for this word
            
    # --- Step 4.7 (Inside Word Loop): Append Final Word Tokens ---
    # Theory: After the inner 'while True' loop finishes, `word_symbols` contains 
    # the final sequence of tokens (characters or subwords) for the current word.
    # We append these tokens to the main `tokenized_output` list using `extend`.
    print("    Step 4.7: Appending final tokens for this word to overall output...")
    tokenized_output.extend(word_symbols)
    print(f"      Appended: {word_symbols}")

print("\nFinished processing all words in the new text.")

Step 4.4: Starting iteration through words of the new text...

  Processing Word: 'alice'
    Step 4.5: Initializing symbols for this word...
      Initial symbols: ['a', 'l', 'i', 'c', 'e', '</w>']
    Step 4.6: Applying learned merges iteratively...
      Applying highest priority merge: ('e', '</w>') (Priority 0) at index 4 -> 'e</w>'
      Updated symbols: ['a', 'l', 'i', 'c', 'e</w>']
      Applying highest priority merge: ('i', 'c') (Priority 13) at index 2 -> 'ic'
      Updated symbols: ['a', 'l', 'ic', 'e</w>']
      Applying highest priority merge: ('ic', 'e</w>') (Priority 31) at index 2 -> 'ice</w>'
      Updated symbols: ['a', 'l', 'ice</w>']
      Applying highest priority merge: ('a', 'l') (Priority 46) at index 0 -> 'al'
      Updated symbols: ['al', 'ice</w>']
      Applying highest priority merge: ('al', 'ice</w>') (Priority 47) at index 0 -> 'alice</w>'
      Updated symbols: ['alice</w>']
      No more applicable learned merges found for this word.
    Step 4.7: Appe

#### Step 4.8: Display Final Tokenization Result

**Theory:** Finally, display the original input text alongside the complete list of tokens produced by applying the learned BPE rules. This demonstrates the outcome of the entire tokenization process.

In [34]:
print("\n--- Final Tokenization Result ---")
print(f"Original Input Text: '{new_text_to_tokenize}'")
print(f"Tokenized Output ({len(tokenized_output)} tokens): {tokenized_output}")


--- Final Tokenization Result ---
Original Input Text: 'Alice thought reading was tiresome without pictures.'
Tokenized Output (21 tokens): ['alice</w>', 'thou', 'g', 'h', 't</w>', 're', 'ad', 'ing</w>', 'was</w>', 'ti', 're', 's', 'o', 'm', 'e</w>', 'wi', 'thou', 't</w>', 'pictures</w>', '.', '</w>']


### Step 5: Conclusion

This notebook provided an extremely detailed, step-by-step, inline implementation of the Byte Pair Encoding algorithm for text tokenization, deliberately avoiding the use of functions or classes to expose the fundamental logic at each stage.

We meticulously followed the key phases:
1.  **Initialization & Preprocessing:** Standardizing the input text (lowercasing), performing initial splitting (words/punctuation), calculating frequencies, representing words as character sequences with crucial `</w>` end-of-word markers, and building the initial character-based vocabulary.
2.  **Iterative Training:** Looping for a predefined number of merges (`num_merges`). In each iteration, we precisely counted adjacent symbol pair frequencies across the *current* corpus state (weighted by word frequency), identified the single most frequent pair, recorded this merge operation along with its priority (based on the iteration number), created a new subword symbol, updated the *entire* corpus representation by replacing the merged pair with the new symbol, and added the new symbol to the vocabulary.
3.  **Inspection:** We examined the final vocabulary size, the list of learned merge rules sorted by the priority in which they were learned, and the final token representation of sample words from the training corpus.
4.  **Tokenization of New Text:** We took an unseen text input, applied the *exact same* preprocessing steps, and then, for each word, iteratively applied the learned merge rules. Crucially, in each tokenization step for a word, we found the merge rule with the *highest priority* (earliest learned) that was applicable to the current sequence of symbols for that word, applied only that merge, and repeated the process until no more learned merges could be applied.

The resulting `tokenized_output` is a sequence of strings, where each string is either an original character or a learned subword from the final BPE vocabulary. This sequence represents the original input text, segmented according to the patterns learned from the training corpus. This detailed breakdown illustrates the core mechanics of BPE, forming the basis for more optimized and complex tokenizers used in modern NLP models.